In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from functions import clean_cycles, clean_sleep_length, add_journal_data, create_features, external_data

In [64]:
cycles = pd.read_csv("data/physiological_cycles.csv")
journals = pd.read_csv("data/journal_entries.csv")
sleeps = pd.read_csv("data/sleeps.csv")
workouts = pd.read_csv("data/workouts.csv")
weather_data = pd.read_csv("data/daily_weather_data.csv")

In [65]:
df1 = clean_cycles(cycles)
df2 = clean_sleep_length(df1)
df3 = add_journal_data(df2, journals)
df4 = create_features(df3)
final_data = external_data(df4, weather_data)

/home/ec2-user/SageMaker/Whoop_Project/functions.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cycles_cleaned['Cycle start time'] = pd.to_datetime(cycles_cleaned['Cycle start time'], dayfirst=True)
/home/ec2-user/SageMaker/Whoop_Project/functions.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cycles_cleaned['Cycle end time'] = pd.to_datetime(cycles_cleaned['Cycle end time'], dayfirst=True)
/home/ec2-user/SageMaker/Whoop_Project/functions.py:13: SettingWithCopyWarning: 
A value is trying to be s

In [66]:
#final_data.to_csv("final_data.csv")

In [67]:
# convert categorical columns to numerical
final_data['Experience bloating?'] = final_data['Experience bloating?'].apply(lambda x: 0 if x == 'FALSE' else 1)
final_data['Experiencing COVID-19 symptoms'] = final_data['Experiencing COVID-19 symptoms'].apply(lambda x: 0 if x == 'FALSE' else 1)
final_data['Experiencing a fever'] = final_data['Experiencing a fever'].apply(lambda x: 0 if x == 'FALSE' else 1)
final_data['Feel energized throughout the day?'] = final_data['Feel energized throughout the day?'].apply(lambda x: 0 if x == 'FALSE' else 1)
final_data['Felt nervous or anxious'] = final_data['Felt nervous or anxious'].apply(lambda x: 0 if x == 'FALSE' else 1)
final_data['Felt recovered'] = final_data['Felt recovered'].apply(lambda x: 0 if x == 'FALSE' else 1)
final_data['Felt you had control over your life'] = final_data['Felt you had control over your life'].apply(lambda x: 0 if x == 'FALSE' else 1)
final_data['Have any alcoholic drinks?'] = final_data['Have any alcoholic drinks?'].apply(lambda x: 0 if x == 'FALSE' else 1)
final_data['Have any caffeine? '] = final_data['Have any caffeine? '].apply(lambda x: 0 if x == 'FALSE' else 1)
final_data['Hydrated sufficiently'] = final_data['Hydrated sufficiently'].apply(lambda x: 0 if x == 'FALSE' else 1)
final_data['Read (non-screened device) while in bed?'] = final_data['Read (non-screened device) while in bed?'].apply(lambda x: 0 if x == 'FALSE' else 1)
final_data['See artificial light upon waking up?'] = final_data['See artificial light upon waking up?'].apply(lambda x: 0 if x == 'FALSE' else 1)
final_data['See direct sunlight upon waking up?'] = final_data['See direct sunlight upon waking up?'].apply(lambda x: 0 if x == 'FALSE' else 1)
final_data['Spend time outdoors?'] = final_data['Spend time outdoors?'].apply(lambda x: 0 if x == 'FALSE' else 1)
final_data['Take creatine?'] = final_data['Take creatine?'].apply(lambda x: 0 if x == 'FALSE' else 1)
final_data['Travel on a plane?'] = final_data['Travel on a plane?'].apply(lambda x: 0 if x == 'FALSE' else 1)
final_data['Viewed a screen device in bed'] = final_data['Viewed a screen device in bed'].apply(lambda x: 0 if x == 'FALSE' else 1)

### create training and testing data sets

In [68]:
y = final_data['Recovery score %']
X = final_data.drop(columns =['Recovery score %','Cycle start time','Cycle end time','Sleep onset','Wake onset','start date','end date','date','Cycle length'])

In [69]:
X = np.array(X).astype('float32')
y = np.array(y).astype('float32')

In [70]:
y = y.reshape(-1,1)

In [71]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42)

In [72]:
#scaling the data before feeding the model
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler_x = StandardScaler()
X_train = scaler_x.fit_transform(X_train)
X_test = scaler_x.transform(X_test)

scaler_y = StandardScaler()
y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)

### set up data in sagemaker

In [46]:
# Boto3 is the Amazon Web Services (AWS) Software Development Kit (SDK) for Python
# Basically it allows us to interface with AWS services like Amazon S3 and Amazon EC2

import sagemaker
import boto3
from sagemaker import Session

# create a Sagemaker session
sagemaker_session = sagemaker.Session()

# define the S3 bucket and prefix that we want to use in this session i.e. where we want to store our data
bucket = bucket = Session().default_bucket()
prefix = 'linear_learner' # prefix is the subfolder within the bucket.

# get the execution role for the notebook instance.
role = sagemaker.get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [47]:
# need to get data into a format that is accepted by AWS

import io # The io module allows for dealing with various types of I/O (text I/O, binary I/O and raw I/O). 
import sagemaker.amazon.common as smac # sagemaker common libary

# Code below converts the data in numpy array format to RecordIO format
# This is the format required by Sagemaker Linear Learner 

buf = io.BytesIO() # create an in-memory byte array (buf is a buffer I will be writing to)
smac.write_numpy_to_dense_tensor(buf, X_train, y_train.reshape(-1))
buf.seek(0) 
# When you write to in-memory byte arrays, it increments 1 every time you write to it
# Let's reset that back to zero 

0

In [48]:
# upload data to S3

import os

# Code to upload RecordIO data to S3

# Key refers to the name of the file
key = 'whoop-train-data'

# The following code uploads the data in record-io format to S3 bucket to be accessed later for training
boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', key)).upload_fileobj(buf)

# Let's print out the training data location in s3
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, prefix, key)
print('uploaded training data location: {}'.format(s3_train_data))

uploaded training data location: s3://sagemaker-eu-west-2-207007680065/whoop_project/train/whoop-train-data


In [49]:
# create an output placeholder in S3 bucket to store the linear learner output

output_location = 's3://{}/{}/output'.format(bucket, prefix)
print('Training artifacts will be uploaded to: {}'.format(output_location))

Training artifacts will be uploaded to: s3://sagemaker-eu-west-2-207007680065/whoop_project/output


### set up linear learner

In [ ]:
# This code is used to get the training container of sagemaker built-in algorithms
# all we have to do is to specify the name of the algorithm, that we want to use

from sagemaker.amazon.amazon_estimator import get_image_uri

container = get_image_uri(boto3.Session().region_name, 'linear-learner')

In [ ]:
# We have pass in the container, the type of instance that we would like to use for training 
# output path and sagemaker session into the Estimator.
# We can also specify how many instances we would like to use for training

linear = sagemaker.estimator.Estimator(container,
                                       role,
                                       train_instance_count = 1,
                                       train_instance_type = 'ml.c4.xlarge',
                                       output_path = output_location,
                                       sagemaker_session = sagemaker_session)


# We can tune parameters
# Train 32 different versions of the model and will get the best out of them (built-in parameters optimization!)

linear.set_hyperparameters(feature_dim = 45,
                           predictor_type = 'regressor',
                           mini_batch_size = 5,
                           epochs = 5,
                           num_models = 32,
                           loss = 'absolute_loss')

# Now we are ready to pass in the training data from S3 to train the linear learner model

linear.fit({'train': s3_train_data})

# Let's see the progress using cloudwatch logs

In [ ]:
# in previous code use train_use_spot_instances = True to save money

### now lets deploy and test the model

In [ ]:
# Deploying the model to perform inference 

linear_regressor = linear.deploy(initial_instance_count = 1,
                                          instance_type = 'ml.m4.xlarge')

In [ ]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

# Content type overrides the data that will be passed to the deployed model, since the deployed model expects data in text/csv format.
# Serializer accepts a single argument, the input data, and returns a sequence of bytes in the specified content type
# Deserializer accepts two arguments, the result data and the response content type, and return a sequence of bytes in the specified content type.

# Set the content type, serializer, and deserializer
linear_regressor.content_type = 'text/csv'
linear_regressor.serializer = CSVSerializer()
linear_regressor.deserializer = JSONDeserializer()

In [ ]:
# making prediction on the test data

result = linear_regressor.predict(X_test)

In [ ]:
result # results are in Json format

In [ ]:
# Since the result is in json format, we access the scores by iterating through the scores in the predictions

predictions = np.array([r['score'] for r in result['predictions']])

In [ ]:
predictions.shape

In [ ]:
y_predict_orig = scaler_y.inverse_transform(predictions.reshape(-1, 1))
y_test_orig = scaler_y.inverse_transform(y_test.reshape(-1, 1))

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from math import sqrt

RMSE = float(format(np.sqrt(mean_squared_error(y_test_orig, y_predict_orig)),'.3f'))
MSE = mean_squared_error(y_test_orig, y_predict_orig)
MAE = mean_absolute_error(y_test_orig, y_predict_orig)
r2 = r2_score(y_test_orig, y_predict_orig)

print('RMSE =',RMSE, '\nMSE =',MSE, '\nMAE =',MAE, '\nR2 =', r2) 

In [ ]:
# Delete the end-point

linear_regressor.delete_endpoint()

### set up data for xgboost

In [73]:
# Convert the array into dataframe in a way that target variable is set as the first column and followed by feature columns
# This is because sagemaker built-in algorithm expects the data in this format.

train_data = pd.DataFrame({'Target': y_train[:,0]})
for i in range(X_train.shape[1]):
    train_data[i] = X_train[:,i]

In [74]:
train_data.to_csv('train.csv', sep=",", header = False, index = False)

In [75]:
import sagemaker
import boto3
from sagemaker import Session

# Let's create a Sagemaker session
sagemaker_session = sagemaker.Session()
bucket = Session().default_bucket() 
prefix = 'XGBoost-Regressor'
key = 'XGBoost-Regressor'
#Roles give learning and hosting access to the data
#This is specified while opening the sagemakers instance in "Create an IAM role"
role = sagemaker.get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [76]:
# read the data from csv file and then upload the data to s3 bucket
import os
with open('train.csv','rb') as f:
    # The following code uploads the data into S3 bucket to be accessed later for training
    boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', key)).upload_fileobj(f)

# Let's print out the training data location in s3
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, prefix, key)
print('uploaded training data location: {}'.format(s3_train_data))

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


uploaded training data location: s3://sagemaker-eu-west-2-207007680065/XGBoost-Regressor/train/XGBoost-Regressor


In [77]:
# creates output placeholder in S3 bucket to store the output

output_location = 's3://{}/{}/output'.format(bucket, prefix)
print('training artifacts will be uploaded to: {}'.format(output_location))

training artifacts will be uploaded to: s3://sagemaker-eu-west-2-207007680065/XGBoost-Regressor/output


### set up xgboost in sagemaker

In [78]:


from sagemaker.amazon.amazon_estimator import image_uris

container = image_uris.retrieve(
    region=boto3.Session().region_name, framework="xgboost", version="1"
)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [79]:
# Specify the type of instance that we would like to use for training 
# output path and sagemaker session into the Estimator. 
# We can also specify how many instances we would like to use for training

# Recall that XGBoost works by combining an ensemble of weak models to generate accurate/robust results. 
# The weak models are randomized to avoid overfitting

# num_round: The number of rounds to run the training.

# Alpha: L1 regularization term on weights. Increasing this value makes models more conservative.

# colsample_by_tree: fraction of features that will be used to train each tree.

# eta: Step size shrinkage used in updates to prevent overfitting. 
# After each boosting step, eta parameter shrinks the feature weights to make the boosting process more conservative.


Xgboost_regressor = sagemaker.estimator.Estimator(container,
                                       role, 
                                       train_instance_count = 1, 
                                       train_instance_type = 'ml.m5.2xlarge',
                                       output_path = output_location,
                                       sagemaker_session = sagemaker_session)

#We can tune the hyper-parameters to improve the performance of the model

Xgboost_regressor.set_hyperparameters(max_depth = 10,
                           objective = 'reg:linear',
                           colsample_bytree = 0.3,
                           alpha = 10,
                           eta = 0.1,
                           num_round = 100
                           )

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [80]:
# Now we are ready to pass in the training data from S3 to train the linear learner model

Xgboost_regressor.fit({'train': s3_train_data})

# Let's see the progress using cloudwatch logs

INFO:sagemaker:Creating training-job with name: xgboost-2023-10-14-14-30-46-311


2023-10-14 14:30:46 Starting - Starting the training job...
2023-10-14 14:31:01 Starting - Preparing the instances for training......
2023-10-14 14:31:49 Downloading - Downloading input data...
2023-10-14 14:32:35 Training - Training image download completed. Training in progress...Arguments: train
[2023-10-14:14:32:53:INFO] Running standalone xgboost training.
[2023-10-14:14:32:53:INFO] Path /opt/ml/input/data/validation does not exist!
[2023-10-14:14:32:53:INFO] File size need to be processed in the node: 0.02mb. Available memory size in the node: 23628.53mb
[2023-10-14:14:32:53:ERROR] Customer Error: Blankspace and colon not found in firstline '0.70786726,-0.65478927,1.8519324,0.079631396,0.021...' of file 'XGBoost-Regressor'. ContentType by defaullt is in libsvm. Please ensure the file is in libsvm format.
Traceback (most recent call last):
  File "/opt/amazon/lib/python3.7/site-packages/sage_xgboost/train.py", line 41, in main
    standalone_train(resource_config, train_config, da

UnexpectedStatusException: Error for Training job xgboost-2023-10-14-14-30-46-311: Failed. Reason: ClientError: Blankspace and colon not found in firstline '0.70786726,-0.65478927,1.8519324,0.079631396,0.021...' of file 'XGBoost-Regressor'. ContentType by defaullt is in libsvm. Please ensure the file is in libsvm format., exit code: 1

### Deploy the xgboost model to perform inference 


In [ ]:
Xgboost_regressor_fit = Xgboost_regressor.deploy(initial_instance_count = 1, instance_type = 'ml.m5.2xlarge')

In [ ]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

# Content type overrides the data that will be passed to the deployed model, since the deployed model expects data in text/csv format.
# Serializer accepts a single argument, the input data, and returns a sequence of bytes in the specified content type
# Deserializer accepts two arguments, the result data and the response content type, and return a sequence of bytes in the specified content type.

# Set the content type, serializer, and deserializer
#Xgboost_regressor_fit.content_type = 'text/csv'
Xgboost_regressor_fit.serializer = CSVSerializer()
#Xgboost_regressor_fit.deserializer = JSONDeserializer()

In [ ]:
# making prediction on the test data

result_XgBoost = Xgboost_regressor_fit.predict(X_test)